# Actuarial Standards of Practice (ASOP) Q&A Machine using Retrieval Augmented Generation (RAG)
This project aims to create a Retrieval-Augmented Generation (RAG) process for actuaries to ask questions on a set of Actuarial Standards of Practice (ASOP) documents. The RAG process utilizes the power of the Large Language Model (LLM) to provide answers to questions on ASOPs.

However, RAG is not without challenges, i.e., hallucination and inaccuracy. This code allows verifiability by providing the context it used to arrive at those answers. This process enables actuaries to validate the information provided by the LLM, empowering them to make informed decisions. By combining the capabilities of LLM with verifiability, this code offers actuaries a robust tool to leverage LLM technology effectively and extract maximum value.

The current example uses either OpenAI's GPT 3.5 turbo AND a local LLM. Note that the current notebook is set up to output results from both models for comparison purposes.  
Using a local LLM can address potential data privacy or security concerns.

View license or further information about the local models used:
- Solar 10.7B Instruct: [cc-by-nc-4.0](https://huggingface.co/upstage/SOLAR-10.7B-Instruct-v1.0) (non-commercial use)
- Mistral 7B Instruct: [Apache License 2.0](https://ollama.com/library/mistral/blobs/sha256:43070e2d4e532684de521b885f385d0841030efa2b1a20bafb76133a5e1379c1)
- [GPT4All embedding model](https://python.langchain.com/docs/integrations/text_embedding/gpt4all)

# 1. Initial Setup
This setup includes loading environment variables from a `.env` file, setting the required environment variables, and importing the necessary modules for further processing. It ensures that the code has access to the required APIs and functions for the subsequent tasks.


In [1]:
# Initial set up
from dotenv import load_dotenv
import os

# Load the variables from .env file and set the API key (or user may manually set the API key)
load_dotenv()  # This loads the variables from .env (not part of repo)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["LANGCHAIN_TRACING_V2"] = "true" # use when you want to debug or monitor the performance of your langchain applications
#os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY') # use when accessing cloud-based language models or services that langchain integrates with

# Import the necessary modules
import bs4
from langchain import hub
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel # for RAG with source
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from IPython.display import display, Markdown, Latex
import glob
import chromadb

In [2]:
# use_OpenAI
embeddings_model = OpenAIEmbeddings()
db_directory = "../data/chroma_db1"
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", 
                 temperature=0) # context window size 16k for GPT 3.5 Turbo

# Open source models used here are for illustration and educational purposes
# Added _ALT at the end of variables to indicate alternative method as a comparison
embeddings_model_ALT = GPT4AllEmbeddings()
db_directory_ALT = "../data/chroma_db2"
# define a local large language model for the augmented generation
# Ollama is one way to easily run inference
#llm = Ollama(model="solar:10.7b-instruct-v1-q5_K_M")
llm_ALT = Ollama(model="mistral:instruct")

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


# 2. Load PDF Files and Convert to a Vector DB
1. Create a function to load and extract text from PDF files in a specified folder. It defines a function called `load_pdfs_from_folder()` that takes a folder path as input and returns a list of extracted text documents from the PDF files in that folder.

2. In the example, the folder path `../data/ASOP` is used, but you can modify it to point to your desired folder.

3. By calling the `load_pdfs_from_folder()` function with the folder path, the code loads the PDF files, extracts the text using the PyPDFLoader, and stores the extracted text documents in the `docs` list.

4. After loading and extracting the text, a `RecursiveCharacterTextSplitter` object is created with specific parameters for chunking the documents. The `split_documents()` method is then used to split the documents into smaller chunks based on the specified parameters.

5. Finally, a Chroma vectorstore is created from the document splits. The vectorstore uses the defined embedding model for embedding the chunks and is saved to the predefined directory.

In [3]:
# Run only when the DB directory is empty
if not os.path.exists(db_directory) or not os.listdir(db_directory):
    # Define a function to load and extract text from PDFs in a folder
    def load_pdfs_from_folder(folder_path):
        # Get a list of PDF files in the specified folder
        pdf_files = glob.glob(f"{folder_path}/*.pdf")
        docs = []
        for pdf_file in pdf_files:
            # Load the PDF file using the PyPDFLoader
            loader = PyPDFLoader(pdf_file) 
            # Extract the text from the PDF and add it to the docs list
            docs.extend(loader.load())
        return docs
    
    # Example folder path
    folder_path = '../data/ASOP'
    
    # Call the function to load and extract text from PDFs in the specified folder
    docs = load_pdfs_from_folder(folder_path)
    
    # Create a text splitter object with specified parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, 
        chunk_overlap=200,
        length_function=len,)
    
    # Split the documents into chunks using the text splitter
    splits = text_splitter.split_documents(docs)
    
    # Create a Chroma vector database from the document splits, using OpenAIEmbeddings for embedding
    vectorstore = Chroma.from_documents(documents=splits, 
                                        embedding=embeddings_model, 
                                        persist_directory=db_directory)

# 3. Retrieve from the Vector DB 

In [3]:
# Get a Chroma vector database with specified parameters
vectorstore = Chroma(embedding_function=embeddings_model, 
                     persist_directory=db_directory)

## Alternative method using local open-source LLM
vectorstore_ALT = Chroma(embedding_function=embeddings_model_ALT, 
                     persist_directory=db_directory_ALT)


In [4]:
## Retrieve and RAG chain

# Create a retriever using the vector database as the search source
retriever = vectorstore.as_retriever(search_type="mmr", 
                                     search_kwargs={'k': 6, 'lambda_mult': 0.25})

## Alternative method using local open-source LLM
retriever_ALT = vectorstore_ALT.as_retriever(search_type="mmr", 
                                     search_kwargs={'k': 6, 'lambda_mult': 0.25})

# Use MMR (Maximum Marginal Relevance) to find a set of documents that are both similar to the input query and diverse among themselves
# Increase the number of documents to get, and increase diversity (lambda mult 0.5 being default, 0 being the most diverse, 1 being the least)

# Load the RAG (Retrieval-Augmented Generation) prompt
prompt = hub.pull("rlm/rag-prompt")

# Define a function to format the documents with their sources and pages
def format_docs_with_sources(docs):
    formatted_docs = "\n\n".join(doc.page_content for doc in docs)
    sources_pages = "\n".join(f"{doc.metadata['source']} (Page {doc.metadata['page'] + 1})" for doc in docs)
    # Added 1 to the page number assuming 'page' starts at 0 and we want to present it in a user-friendly way

    return f"Documents:\n{formatted_docs}\n\nSources and Pages:\n{sources_pages}"

# Create a RAG chain using the formatted documents as the context
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs_with_sources(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

# Create a parallel chain for retrieving and generating answers
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

## Alternative method using local open-source LLM
# Create a RAG chain using the formatted documents as the context
rag_chain_from_docs_ALT = (
    RunnablePassthrough.assign(context=(lambda x: format_docs_with_sources(x["context"])))
    | prompt
    | llm_ALT
    | StrOutputParser()
)

# Create a parallel chain for retrieving and generating answers
rag_chain_with_source_ALT = RunnableParallel(
    {"context": retriever_ALT, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs_ALT)


# 4. Generate Q&A

In [11]:
def generate_output():
    # Prompt the user for a question on ASOP
    usr_input = input("What is your question on ASOP?: ")

    # Invoke the RAG chain with the user input as the question
    output = rag_chain_with_source.invoke(usr_input)
    output_ALT = rag_chain_with_source_ALT.invoke(usr_input)

    # Generate the Markdown output with the question, answer, and context
    markdown_output = "### Question\n{}\n\n### Open AI Answer\n{}\n\n".format(output['question'], output['answer'])
    markdown_output += "### Local LLM Answer\n{}\n\n### Open AI Context\n".format(output_ALT['answer'])

    last_page_content = None  # Variable to store the last page content
    i = 1 # Source indicator

    # Iterate over the context documents to format and include them in the output
    for doc in output['context']:
        current_page_content = doc.page_content.replace('\n', '  \n')  # Get the current page content
        
        # Check if the current content is different from the last one
        if current_page_content != last_page_content:
            markdown_output += "- **Open AI Source {}**: {}, page {}:\n\n{}\n".format(i, doc.metadata['source'], doc.metadata['page'], current_page_content)
            i = i + 1
        last_page_content = current_page_content  # Update the last page content

    markdown_output += "\n\n### Local LLM Context\n"

    last_page_content = None  # Variable to store the last page content
    i = 1 # Source indicator

    # Iterate over the context documents to format and include them in the output
    for doc in output_ALT['context']:
        current_page_content = doc.page_content.replace('\n', '  \n')  # Get the current page content
        
        # Check if the current content is different from the last one
        if current_page_content != last_page_content:
            markdown_output += "- **Local LLM Source {}**: {}, page {}:\n\n{}\n".format(i, doc.metadata['source'], doc.metadata['page'], current_page_content)
            i = i + 1
        last_page_content = current_page_content  # Update the last page content

    
    # Display the Markdown output
    display(Markdown(markdown_output))

### Example questions related to ASOPs
- explain ASOP No. 14
- How are expenses relfected in cash flow testing based on ASOP No. 22?
- What is catastrophe risk?
- When do I update assumptions?
- What should I do when I do not have credible data to develop non-economic assumptions?

In [13]:
generate_output()

What is your question on ASOP?:  What should I do when I do not have credible data to develop non-economic assumptions?


### Question
What should I do when I do not have credible data to develop non-economic assumptions?

### Open AI Answer
When credible data is not available to develop non-economic assumptions, consider other information sources such as pricing practices in the insurance business or experience of other insurance companies. Obtain input from individuals with relevant expertise and give weight to their input when developing assumptions. If necessary, make judgmental adjustments or assumptions to the data, disclose them, and consider the potential uncertainty or bias in the results.

### Local LLM Answer
 When you don't have credible data to develop non-economic assumptions, consider the following steps based on the provided context from ASOP No. 23 and ASOP No. 25:

1. Review the overall data quality to determine if it is sufficient for completing the assignment (ASOP No. 23). If necessary, make assumptions based on relevant available data, such as averages or industry benchmarks.
2. Ensure consistency among all selected assumptions for a particular measurement and assess their appropriateness in the current model run (ASOP No. 23 and ASOP No. 056).
3. Assess the reasonability of the model output when determining if assumptions are reasonable in aggregate, recognizing the legitimacy of benefit plan experience (ASOP No. 25).
4. Evaluate data quality at each level of usage as you combine or separate data for your analysis (ASOP No. 23).
5. Consider alternative data sources and elements, taking into account their availability and relevance to the purpose of your analysis (ASOP No. 23).

### Open AI Context
- **Open AI Source 1**: ../data/ASOP/asop019_137.pdf, page 7:

information is available. When experience of the business is unavailable or insufficient to   
provide a credible basis on which to develo p assumptions, the actuary should consider   
other information sources in setting assumptions. Other information sources may include the pricing or reserving practices applicable  to the insurance business and the available   
experience of other insurance businesses with comparable policies or contracts, markets,   
and operating environment.    
  In developing assumptions for which the actuary believes additional expertise is needed,   
the actuary should obtain necessary input from persons possessing the relevant   
knowledge or expertise, and should gi ve due weight to their input.    
  When setting assumptions for use in an appraisal, the actuary should take reasonable   
steps to ensure that each set of assump tions used is internally consistent.   
   
3.4 Discount Rate  
⎯If the appraisal is based on the di scounted value of projected earnings,
- **Open AI Source 2**: ../data/ASOP/asop027_197.pdf, page 20:

study. For each previously selected assumption that the actuary determines is no longer   
reasonable, the actuary should select a reasonable new assumption.    
   
3.14 Assessing Assumptions Not Selected by the Actuary—At each measurement date , the   
actuary should assess the reasonableness of each economic assumption that the actuary has   
not selected (other than prescribed assumptions or methods set by law  or assumptions   
disclosed in accordance with section 4.2[b]), using the guidance set forth in this standard   
to the extent practicable.   
   
3.15 Phase-In of Changes in Assumptions—If an  economic assumption is being phased in over   
a period that includes multiple measurement dates , the actuary should determine the   
reasonableness of the  economic assumption and its consistency with other assumptions as   
of the measurement date  at which it is applied, without regard to changes to the   
assumption planned for future measurement dates . If the actuary determines that an
- **Open AI Source 3**: ../data/ASOP/asop013_133.pdf, page 8:

for most property/casualty insuranc e plans or policies. In such procedures, actuaries generally   
place reliance on (1) data generated by the book of  business being analyzed, (2) other insurance   
data, and (3) non-insurance data, in that order of preference.  Mathematical techniques are often   
used to smooth and extrapolate from historical data. In the absence of strong contrary   
indications, there is a reliance on extrapolations of historical in surance data. Procedures based on   
non-insurance data are also use d. In trending procedures, judgmen tal considerations generally   
include, but are not limited to, th e historical data used, the succe ss of these techniques in making   
prior projections, the statistical goodness of fit of the techniques to the historical data, and the   
impact of any sudden, nonrecurring changes (for example, tort re form) which had not yet been   
incorporated in the historical data.
- **Open AI Source 4**: ../data/ASOP/asop023_185.pdf, page 8:

obtain additional or corrected data  that will allow the analysis to be performed;   
   
c. judgmental adjustments or assu mptions can be applied to the data  that allow the   
actuary to perform the analysis. Any judgmental adjustments to data  or assumptions   
should be disclosed in acco rdance with section 4.1(f). If the actuary judges that the   
use of the data , even with adjustments and a ssumptions applied, may cause the   
results to be highly uncertain  or contain a significant bias, the actuary may choose to   
complete the assignment but should disclose the potential existence of the uncertainty   
or bias, and, if reasonably determinable, the nature and potentia l magnitude of such   
uncertainty or bias, in accordance with se ction 4.1(g). Alterna tively, the actuary may   
compensate for the data  deficiencies by adjusting the results, such as by increasing   
the range of reasonable estimates, and disclose the adjustments, in accordance with section 4.1(f);
- **Open AI Source 5**: ../data/ASOP/asop051_188.pdf, page 24:

ASOP No. 51—September 2017   
   
   
 18Section 3.4, Assumptions for Assess ment of Risk (now  section 3.5)   
Comment   
   
   
Response One commentator suggested that empirical data be used to select assumptions rather than using   
professional judgment.   
   
The reviewers note that the section Assumptions for Assessment of Risk reads “The assumptions used for assessment of risk may be based on economic and demographic data and analyses,” but   
believe “the actuary should use professional judgment in selecting [these] assumptions” and made   
no change in response to this comment.   
Comment    
   
Response Two commentators suggested the term “plausible” is  not clear and also that implausible outcomes   
should be considered.   
   
The reviewers believe the term “plausible,” combined with the requirement for the actuary to use professional judgment, is appropriate for this standard and made no change in response to this   
comment.   
Comment
- **Open AI Source 6**: ../data/ASOP/asop046_165.pdf, page 12:

ASOP No. 46—September 2012    
   
 7b. prices in the marketplace;   
   
c. opinions of other experts;   
   
d. the fit of the assumed dist ribution to available data;   
   
e. the ability of the assumed distribution to reflect possible extreme values;    
   
f. sensitivity of results to changes in assumptions;    
   
g.  internal consistency of the assumptions; and  h. consistency in the app lication of assumptions.   
 3.3.5   Validation of the Economic Capital Model  
—Economic capital is often   
determined based on the resu lts of stochastic models that produce a large number   
of outcomes. The actuary should devise a ppropriate tests of the distribution of   
outcomes calculated by the model (for example, in comparison to the range of results in similar models or to histori cal outcomes over time) and the sensitivity of   
those distributions to changes in the assumptions and parameters. The actuary should also perform validation tests to determine whether the model results are


### Local LLM Context
- **Local LLM Source 1**: ../data/ASOP/asop035_198.pdf, page 18:

elements such as birth dates or hire dates. Accordingly, assumptions for missing or   
incomplete data may be necessary if the actuary has determined, in accordance with   
ASOP No. 23, Data Quality , that the overall data are of sufficient quality to   
complete the assignment. Data actually supplied may be relevant in making such   
assumptions. For example, it may be appropriate to assume a missing birth date is   
equal to the average birth date for other participants who have complete data and   
who have the same service credits as the participant whose date of birth is missing.   
   
3.6 Consistency among Assumptions Selected by the Actuary for a Particular Measurement—  
With respect to a particular measurement, the actuary should select demographic   
assumptions  that are consistent with the other assumptions selected by the actuary,   
including economic assumptions, unless an assumption considered individually is not
- **Local LLM Source 2**: ../data/ASOP/asop056_195.pdf, page 11:

identifying the possibility of  an inconsistency with other assumptions .    
   
d. Appropriateness of Input in Curre nt Model Run—Where practical a nd   
appropriate, the actuary reusing an existing model  should evaluate whether   
input unchanged from a prior model run is still appropriate  for use in the   
current model run . For example, models  used in financial reporting may   
offer opportunities to compare assumptions  to emerging experience in the   
aggregate.    
   
e. Reasonable Model in the Aggregate—The actuary should assess the    
reasonability of the model output  when determining whether the   
assumptions  are reasonable in the aggregate. While assumptions  might
- **Local LLM Source 3**: ../data/ASOP/asop006_177.pdf, page 30:

plan experience relative to normative  ranges of value but also recognize   
the legitimacy of the benefit plan  experience, to the extent it is credible,   
and the limitations of applying normative data to an unrelated situation.   
ASOP No. 25 provides guidance in the assignment of credibility values to   
data.   
     c. Data Quality at Each Level of Usage—Data that may be of appropriate   
quality for determination of certain assumptions within a model may not   
be of appropriate quality for determ ination of other assumptions. When   
data are combined or separated, the actuary should review the data for   
suitability for the purpose. For example, data from a benefit plan  may be   
sufficient for setting an aggregate per ca pita health care cost but not be of   
sufficient size to set per capita health care costs by location.    
  3.10 Administrative Inconsistencies—In general, the actuary may rely on the plan sponsor’s
- **Local LLM Source 4**: ../data/ASOP/asop023_185.pdf, page 7:

ASOP No. 23—Doc. No. 185    
   
3   
   
 sets or data  sources, if any, to be considere d. The actuary should do the following:   
   
a.        consider the data elements  that are desired and possible alternative data elements ;   
and   
   
b.        select the data  for the analysis with consideration of the following:   
   
1. whether the data  constitute appropriate data , including whether the data  are   
sufficiently current;    
   
2. whether the data  are reasonable with particular attention to internal   
consistency;   
   
3. whether the data  are reasonable given relevant external information that is   
readily available and known to the actuary;   
   
4. the degree to which the data  are sufficient ;   
   
5. any known significant limitations of the  data ;   
   
6. the availability of additional or alternative data  and the benefit to be gained   
from such additional or alternative data , balanced against how practical it is to   
collect and compile such additional or alternative data ; and
- **Local LLM Source 5**: ../data/ASOP/asop035_198.pdf, page 12:

rather than selecting a separate assumption for each.   
   
 3.2.2 Consider the Relevant Assumption Universe—The actuary should be familiar with   
the assumption universe  relevant to each type of assumption identified in section   
3.2.1. The assumption universe  may include tables or factors particular to the   
given plan as well as general tables, factors, and modifications to the tables that are   
available to the actuary. Sources of information relevant to demographic   
assumptions  may include the following:   
   
  a. experience studies or published tables based on experience under uninsured   
plans and annuity contracts, or based on any other populations considered   
representative of the group at hand;   
   
  b. relevant plan or plan sponsor experience, which may include analyses of   
gains or losses by source;    
   
  c. studies or reports of the effects of plan design, specific events (for example,   
shutdown), economic conditions, or sponsor characteristics on the
- **Local LLM Source 6**: ../data/ASOP/asop022_203.pdf, page 10:

business . For example, mortality improvement may be different   
between life and annuity products ;   
   
b. the source and credibility of the data from which the assumptions    
are derived (f or further guidance, the actuary should refer to ASOP   
No. 23, Data Quality , and ASOP No. 25, Credibility Procedures ).   
For example, different trends may be appropriate when using   
company experience vs. industry studies ; and    
   
c. the impact of trends on cash flows . For example, the effect of   
future economic conditions on policyholder elections .   
   
3.1.2.2 Margins —The actuary should consider including margins in assumptions   
to reflect adverse deviation . When determining the level of  assumption   
margins , if any, the actuary should take into account the following:    
   
a. the level of uncertainty for the assumption, including sparsity of   
data;


# 5. References
- https://www.actuarialstandardsboard.org/standards-of-practice/
- https://python.langchain.com/docs/use_cases/question_answering/quickstart
- https://python.langchain.com/docs/use_cases/question_answering/sources
- https://python.langchain.com/docs/integrations/text_embedding/
- https://docs.gpt4all.io/gpt4all_python_embedding.html#gpt4all.gpt4all.Embed4All
- https://chat.langchain.com/